In [2]:
import yfinance as yf
import pandas as pd

# Fetch historical stock data using yfinance
def fetch_data(stock_symbol):
    # Define the stock symbol and the period for which we want the data
    period = "1d"  # Last 7 days
    interval = "1m"  # 1-minute intervals

    # Use yfinance to download the data
    stock_data = yf.download(stock_symbol, period=period, interval=interval)
    return stock_data

# Process the data to create features and labels
def prepare_data(stock_data, look_forward=10):
    # Calculate the future price after `look_forward` minutes (our label)
    stock_data['Future_Price'] = stock_data['Close'].shift(-look_forward)

    # Drop the last `look_forward` rows since they don't have a future price
    stock_data = stock_data[:-look_forward]

    return stock_data

In [3]:
from sklearn.preprocessing import MinMaxScaler

def normalize_data(data):
    scaler = MinMaxScaler(feature_range=(0, 1))
    data[['Close', 'Volume', 'Future_Price']] = scaler.fit_transform(data[['Close', 'Volume', 'Future_Price']])
    return data, scaler

def split_data(data, test_size=0.2):
    # Calculate the index to split the data
    split_idx = int(len(data) * (1 - test_size))
    train = data[:split_idx]
    test = data[split_idx:]
    
    return train, test

import numpy as np

def create_sequences(data, sequence_length=60):
    xs = []
    ys = []
    for i in range(len(data) - sequence_length):
        x = data[['Close', 'Volume']].iloc[i:(i + sequence_length)].values
        y = data['Future_Price'].iloc[i + sequence_length]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

def build_model(input_shape):
    model = Sequential()
    model.add(LSTM(50, return_sequences=True, input_shape=input_shape))
    model.add(LSTM(50, return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(25))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

2024-03-15 02:15:23.396030: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
nifty_500df = pd.read_csv('ind_nifty500list.csv')
nifty_500df.head()

,Company Name,Industry,Symbol,Series,ISIN Code
0,360 ONE WAM Ltd.,Financial Services,360ONE,EQ,INE466L01038
1,3M India Ltd.,Diversified,3MINDIA,EQ,INE470A01017
2,ABB India Ltd.,Capital Goods,ABB,EQ,INE117A01022
3,ACC Ltd.,Construction Materials,ACC,EQ,INE012A01025
4,AIA Engineering Ltd.,Capital Goods,AIAENG,EQ,INE212H01026


In [5]:
for stock_symbol in nifty_500df['Symbol']:

    raw_data = fetch_data(stock_symbol+".NS")
    prepared_data = prepare_data(raw_data)

    prepared_data, scaler = normalize_data(prepared_data)
    train_data, test_data = split_data(prepared_data)

    sequence_length = 60  # number of minutes to look back
    X_train, y_train = create_sequences(train_data, sequence_length)
    X_test, y_test = create_sequences(test_data, sequence_length)
    
    model = build_model((X_train.shape[1], X_train.shape[2]))

    history = model.fit(
    X_train, y_train, 
    epochs=5, 
    batch_size=32, 
    validation_data=(X_test, y_test)
    )

    model.save("models/"+stock_symbol+".h5")

[*********************100%%**********************]  1 of 1 completed
/var/folders/cx/8v11zwh97cxgdgt_ph13_56r0000gn/T/ipykernel_70775/1842824869.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[['Close', 'Volume', 'Future_Price']] = scaler.fit_transform(data[['Close', 'Volume', 'Future_Price']])
2024-03-15 02:15:37.992813: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-03-15 02:15:37.992871: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical Plug

Epoch 1/5


2024-03-15 02:15:44.706181: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-15 02:15:45.056006: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Adam/AssignAddVariableOp.
2024-03-15 02:15:46.254163: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-15 02:15:47.984906: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-15 02:15:52.971130: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-15 02:15:54.985255: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


8/8 [==============================] - ETA: 0s - loss: 0.1095

2024-03-15 02:16:04.488146: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-15 02:16:04.717433: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-15 02:16:05.409294: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


...

8/8 [==============================] - ETA: 0s - loss: 0.1196

2024-03-15 02:16:34.368443: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-15 02:16:34.609381: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-15 02:16:35.380117: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


8/8 [==============================] - 12s 941ms/step - loss: 0.1196 - val_loss: 0.1697
Epoch 2/5
8/8 [==============================] - 1s 110ms/step - loss: 0.0244 - val_loss: 0.2122
Epoch 3/5
8/8 [==============================] - 1s 106ms/step - loss: 0.0141 - val_loss: 0.2581
Epoch 4/5
8/8 [==============================] - 1s 112ms/step - loss: 0.0124 - val_loss: 0.1883
Epoch 5/5
8/8 [==============================] - 1s 108ms/step - loss: 0.0121 - val_loss: 0.1959


/Users/hbot07/Library/Python/3.9/lib/python/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
[*********************100%%**********************]  1 of 1 completed
/var/folders/cx/8v11zwh97cxgdgt_ph13_56r0000gn/T/ipykernel_70775/1842824869.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[['Close', 'Volume', 'Future_Price']] = scaler.fit_transform(data[['Close', 'Volume', 'Future_Price']])


KeyboardInterrupt: 